# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a Mall in the city of Chennai, Tamil Nadu, India_**
- Build a dataframe of neighborhoods in Chennai, Tamil Nadu by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall
***
### 1. Import libraries

In [43]:
!pip install geocoder
!pip install folium
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [44]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Chennai").text

In [45]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [46]:
# create a list to store neighborhood data
neighbourhoodList = []

In [47]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighbourhoodList.append(row.text)

In [49]:
# create a new DataFrame from the list
c_df = pd.DataFrame({"Neighbourhood": neighbourhoodList})

c_df.head()

,Neighbourhood
0,Alandur
1,Anna Nagar
2,"Ashok Nagar, Chennai"
3,Assisi Nagar
4,Ayanavaram


In [50]:
# print the number of rows of the dataframe
c_df.shape

(68, 1)

### 3. Get the geographical coordinates

In [51]:
# define a function to get coordinates
def get_latlng(neighbourhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, Tamil Nadu'.format(neighbourhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [53]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighbourhood) for neighbourhood in c_df["Neighbourhood"].tolist() ]

In [54]:
coords

[[13.00013000000007, 80.20060000000007],
 [13.083590000000072, 80.21020000000004],
 [13.035390000000064, 80.21220000000005],
 [13.164570000000026, 80.23274000000004],
 [13.09883000000002, 80.23238000000003],
 [13.074621704974275, 80.24277657646144],
 [12.932770000000062, 80.14387000000005],
 [12.95234000000005, 80.14411000000007],
 [12.988610000000051, 80.15100000000007],
 [12.82725000000005, 80.22866000000005],
 [12.837900000000047, 80.05327000000005],
 [13.040920000000028, 80.13649000000004],
 [13.11035000000004, 80.21301000000005],
 [13.129720000000077, 80.18300000000005],
 [13.120580000000075, 80.06047000000007],
 [12.956150000000036, 80.17885000000007],
 [12.793410000000051, 80.22010000000006],
 [13.081980000000044, 80.24448000000007],
 [13.051520000000039, 80.22421000000008],
 [13.136630000000025, 80.24479000000008],
 [13.131830000000036, 80.19928000000004],
 [13.096050000000048, 80.05292000000009],
 [13.116800000000069, 80.27726000000007],
 [13.183260000000075, 80.24059000000005

In [55]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [56]:
# merge the coordinates into the original dataframe
c_df['Latitude'] = df_coords['Latitude']
c_df['Longitude'] = df_coords['Longitude']

In [57]:
# check the neighbourhoods and the coordinates
print(c_df.shape)
c_df

(68, 3)


,Neighbourhood,Latitude,Longitude
0,Alandur,13.000130,80.200600
1,Anna Nagar,13.083590,80.210200
2,"Ashok Nagar, Chennai",13.035390,80.212200
3,Assisi Nagar,13.164570,80.232740
4,Ayanavaram,13.098830,80.232380
5,Chennai city,13.074622,80.242777
6,Chitlapakkam,12.932770,80.143870
7,Chromepet,12.952340,80.144110
8,Cowl Bazaar,12.988610,80.151000
9,Egattur (Kanchipuram District),12.827250,80.228660


In [58]:
# save the DataFrame as CSV file
c_df.to_csv("c_df.csv", index=False)

### 4. Create a map of Chennai with neighborhoods superimposed on top

In [59]:
# get the coordinates of Chennai
address = 'Chennai, Tamil Nadu'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai, Tamil Nadu {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai, Tamil Nadu 13.0801721, 80.2838331.


In [61]:
# create map of Chennai using latitude and longitude values
map_c = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(c_df['Latitude'], c_df['Longitude'], c_df['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_c)  
    
map_c

In [62]:
# save the map as HTML file
map_c.save('map_c.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [63]:
# define Foursquare Credentials and Version
CLIENT_ID = 'WP2OHTK5GACF52PF3WO1NWMRTIV1XFHU5PT2CMKENEPVFYID' # your Foursquare ID
CLIENT_SECRET = 'UKEBGWG21TALEFQ3KJ1UEE4V0OWZVY4NLXYSAIGZFIRY1MNH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WP2OHTK5GACF52PF3WO1NWMRTIV1XFHU5PT2CMKENEPVFYID
CLIENT_SECRET:UKEBGWG21TALEFQ3KJ1UEE4V0OWZVY4NLXYSAIGZFIRY1MNH


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [64]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighbourhood in zip(c_df['Latitude'], c_df['Longitude'], c_df['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighbourhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [65]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1311, 7)


,Neighbourhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alandur,13.00013,80.2006,Sukkkubai Beef Biryani Shop,12.998769,80.201381,Indian Restaurant
1,Alandur,13.00013,80.2006,Pizza Republic,12.990987,80.198613,Pizza Place
2,Alandur,13.00013,80.2006,Moon & Six Pence - The Irish Bar,13.007848,80.208152,Bar
3,Alandur,13.00013,80.2006,Q Bar,13.016606,80.204853,Restaurant
4,Alandur,13.00013,80.2006,Hilton,13.016621,80.204787,Hotel


**Let's check how many venues were returned for each neighorhood**

In [66]:
venues_df.groupby(["Neighbourhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhood,,,,,,
Alandur,45,45,45,45,45,45
Anna Nagar,100,100,100,100,100,100
"Ashok Nagar, Chennai",69,69,69,69,69,69
Assisi Nagar,4,4,4,4,4,4
Ayanavaram,26,26,26,26,26,26
Chennai city,100,100,100,100,100,100
Chitlapakkam,16,16,16,16,16,16
Chromepet,16,16,16,16,16,16
Cowl Bazaar,18,18,18,18,18,18


**Let's find out how many unique categories can be curated from all the returned venues**

In [67]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 151 uniques categories.


In [68]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Pizza Place', 'Bar', 'Restaurant', 'Hotel',
       'Church', 'Train Station', 'Asian Restaurant', 'Ice Cream Shop',
       'Donut Shop', 'South Indian Restaurant', "Men's Store",
       'Breakfast Spot', 'Bakery', 'Italian Restaurant',
       'Multicuisine Indian Restaurant', 'Pool Hall',
       'Japanese Restaurant', 'Café', 'Bus Line', 'Juice Bar',
       'Fast Food Restaurant', 'Department Store', 'Metro Station',
       'Cafeteria', 'Gym Pool', 'Sandwich Place', 'Gym', 'Park',
       'Chinese Restaurant', 'Snack Place', 'Coffee Shop',
       'Vegetarian / Vegan Restaurant', 'Shoe Store', 'Indian Sweet Shop',
       'American Restaurant', 'Burger Joint', 'Middle Eastern Restaurant',
       'Shopping Mall', 'Market', 'BBQ Joint', 'New American Restaurant',
       'Clothing Store', 'Paper / Office Supplies Store', 'Jewelry Store',
       'Multiplex', 'Furniture / Home Store', 'Farmers Market',
       'Bookstore', 'Bistro'], dtype=object)

In [69]:
# check if the results contain "Shopping Mall"
"Neighbourhood" in venues_df['VenueCategory'].unique()

False

### 6. Analyze Each Neighborhood

In [70]:
# one hot encoding
c_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
c_onehot['Neighbourhoods'] = venues_df['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [c_onehot.columns[-1]] + list(c_onehot.columns[:-1])
c_onehot = c_onehot[fixed_columns]

print(c_onehot.shape)
c_onehot.head()

(1311, 152)


,Neighbourhoods,ATM,Accessories Store,Afghan Restaurant,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Beach,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gourmet Shop,Grocery Store,Gym,Gym Pool,Hookah Bar,Hospital,Hotel,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Light Rail Station,Lounge,Malay Restaurant,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store,Park,Persian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Club,Supermarket,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [72]:
c_grouped = c_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(c_grouped.shape)
c_grouped

(68, 152)


,Neighbourhoods,ATM,Accessories Store,Afghan Restaurant,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Beach,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gourmet Shop,Grocery Store,Gym,Gym Pool,Hookah Bar,Hospital,Hotel,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Light Rail Station,Lounge,Malay Restaurant,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store,Park,Persian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Club,Supermarket,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Alandur,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.022222,0.00,0.000000,0.000000,0.022222,0.022222,0.000000,0.000000,0.0,0.00,0.00000,0.000000,0.00000,0.044444,0.00000,0.000000,0.022222,0.000000,0.000000,0.022222,0.066667,0.000000,0.000000,0.00,0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.022222,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.022222,0.00,0.0,0.111111,0.000000,0.000000,0.022222,0.177778,0.00,0.000000,0.000000,0.022222,0.00,0.022222,0.000000,0.022222,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.022222,0.022222,0.00,0.000000,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.044444,0.000000,0.000000,0.022222,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.044444,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.044444,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Anna Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.010000,0.00,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.0,0.01,0.00000,0.010000,0.00000,0.000000,0.00000,0.010000,0.000000,0.020000,0.000000,0.000000,0.040000,0.000000,0.030000,0.00,0.000000,0.040000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.020000,0.010000,0.100000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.0,0.010000,0.000000,0.000000,0.020000,0.130000,0.01,0.000000,0.00000

In [73]:
len(c_grouped[c_grouped["Shopping Mall"] > 0])

12

**Create a new DataFrame for Shopping Mall data only**

In [74]:
c_mall = c_grouped[["Neighbourhoods","Shopping Mall"]]

In [75]:
c_mall.head()

,Neighbourhoods,Shopping Mall
0,Alandur,0.000000
1,Anna Nagar,0.020000
2,"Ashok Nagar, Chennai",0.014493
3,Assisi Nagar,0.000000
4,Ayanavaram,0.000000


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [76]:
# set number of clusters
kclusters = 3

c_clustering = c_mall.drop(["Neighbourhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(c_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 0, 0, 2, 0, 1, 0, 1], dtype=int32)

In [77]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood.
c_merged = c_mall.copy()

# add clustering labels
c_merged["Cluster Labels"] = kmeans.labels_

In [78]:
c_merged.rename(columns={"Neighbourhoods": "Neighbourhood"}, inplace=True)
c_merged.head()

,Neighbourhood,Shopping Mall,Cluster Labels
0,Alandur,0.000000,0
1,Anna Nagar,0.020000,2
2,"Ashok Nagar, Chennai",0.014493,2
3,Assisi Nagar,0.000000,0
4,Ayanavaram,0.000000,0


In [79]:
# merge chennai_grouped with chennai_data to add latitude/longitude for each neighbourhood
c_merged = c_merged.join(c_df.set_index("Neighbourhood"), on="Neighbourhood")

print(c_merged.shape)
c_merged.head() # check the last columns!

(68, 5)


,Neighbourhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.00013,80.20060
1,Anna Nagar,0.020000,2,13.08359,80.21020
2,"Ashok Nagar, Chennai",0.014493,2,13.03539,80.21220
3,Assisi Nagar,0.000000,0,13.16457,80.23274
4,Ayanavaram,0.000000,0,13.09883,80.23238


In [80]:
# sort the results by Cluster Labels
print(c_merged.shape)
c_merged.sort_values(["Cluster Labels"], inplace=True)
c_merged

(68, 5)


,Neighbourhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.000130,80.200600
34,Nazarethpettai,0.000000,0,13.037100,80.057550
35,Oragadam,0.000000,0,13.137440,80.153830
36,Padappai,0.000000,0,12.886500,80.021550
37,Palavakkam,0.000000,0,12.961390,80.257430
38,Pallavaram,0.000000,0,12.974440,80.148520
39,Pallikaranai,0.000000,0,12.955670,80.220800
41,Panambakkam,0.000000,0,13.077610,80.155830
42,Pattabiram,0.000000,0,13.123330,80.059440
43,Peerkankaranai,0.000000,0,12.912240,80.098950


**Finally, let's visualize the resulting clusters**

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(c_merged['Latitude'], c_merged['Longitude'], c_merged['Neighbourhood'], c_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [83]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [84]:
c_merged.loc[c_merged['Cluster Labels'] == 0]

,Neighbourhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.0,0,13.00013,80.20060
34,Nazarethpettai,0.0,0,13.03710,80.05755
35,Oragadam,0.0,0,13.13744,80.15383
36,Padappai,0.0,0,12.88650,80.02155
37,Palavakkam,0.0,0,12.96139,80.25743
38,Pallavaram,0.0,0,12.97444,80.14852
39,Pallikaranai,0.0,0,12.95567,80.22080
41,Panambakkam,0.0,0,13.07761,80.15583
42,Pattabiram,0.0,0,13.12333,80.05944
43,Peerkankaranai,0.0,0,12.91224,80.09895


#### Cluster 1

In [85]:
c_merged.loc[c_merged['Cluster Labels'] == 1]

,Neighbourhood,Shopping Mall,Cluster Labels,Latitude,Longitude
7,Chromepet,0.062500,1,12.95234,80.14411
33,Navalur,0.058824,1,12.84584,80.22648
9,Egattur (Kanchipuram District),0.052632,1,12.82725,80.22866
40,Pammal,0.043478,1,12.96814,80.13359
55,Semmencherry,0.050000,1,12.86557,80.22051


#### Cluster 2

In [86]:
c_merged.loc[c_merged['Cluster Labels'] == 2]

,Neighbourhood,Shopping Mall,Cluster Labels,Latitude,Longitude
5,Chennai city,0.010000,2,13.074622,80.242777
17,Kilpauk,0.017857,2,13.081980,80.244480
2,"Ashok Nagar, Chennai",0.014493,2,13.035390,80.212200
1,Anna Nagar,0.020000,2,13.083590,80.210200
64,Vadapalani,0.015152,2,13.052260,80.211200
65,Virugambakkam,0.017241,2,13.055900,80.193490
18,Kodambakkam,0.020000,2,13.051520,80.224210


#### Observations:
Most of the shopping malls are concentrated in the central area of Chennai city, with the highest number in cluster 0 and moderate number in cluster 1. On the other hand, cluster 2 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 which already have high concentration of shopping malls and suffering from intense competition.